In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os
import tensorflow as tf
import numpy as np

# Set the seed for random operations. 
# This let our experiments to be reproducible. 
SEED = 1234
tf.random.set_seed(SEED)  

# Set GPU memory growth 
# Allows to only as much GPU memory as needed
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

In [ ]:
# ImageDataGenerator
# ------------------

from tensorflow.keras.preprocessing.image import ImageDataGenerator

apply_data_augmentation = True

# Create training ImageDataGenerator object
if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(rotation_range=30,
                                        width_shift_range=0.2,
                                        height_shift_range=0.2,
                                        brightness_range=(0.6,1),
                                        channel_shift_range=150,
                                        shear_range=0.2,
                                        zoom_range=0.2,
                                        horizontal_flip=True,
                                        vertical_flip=True,
                                        fill_mode='nearest',
                                        rescale=1./255)
else:
    train_data_gen = ImageDataGenerator(rescale=1./255)

# Create validation and test ImageDataGenerator objects
valid_data_gen = ImageDataGenerator(rescale=1./255)
#test_data_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
import shutil, random, json

#delete file that fill the memory
useless_path = '/kaggle/working/classification_experiments'
if os.path.exists(useless_path):
    shutil.rmtree(useless_path)

#check if the tmp folder exist
path = '/kaggle/working/tmp'
if os.path.exists(path):
    shutil.rmtree(path)
if not os.path.exists(path):
    os.mkdir(path)
if not os.path.exists(path+'/training'):
    os.mkdir(path+'/training')
if not os.path.exists(path+'/validation'):
    os.mkdir(path+'/validation')

# Source path 
source = "/kaggle/input/ann-and-dl-image-classification/Classification_Dataset/training"

# Destination path 
dest_train = path+'/training'
dest_valid = path+'/validation'

# dictionary with the format shown in the Evaluation tab
dataset_split = {}  
dataset_split["training"] = {}
dataset_split["validation"] = {}

#create train and validation into the tmp folder
for folder in os.listdir(source):
    if not os.path.exists(dest_train+'/'+folder):
        os.mkdir(dest_train+'/'+folder)
    if not os.path.exists(dest_valid+'/'+folder):
        os.mkdir(dest_valid+'/'+folder)
    
    dataset_split["training"][folder] = []         # create the list of images used in training set for each class
    dataset_split["validation"][folder] = []         # create the list of images used in validation set for each class
    cl = source+'/'+folder                             # create path of the class
    files = os.listdir(cl)                            # list of files for the class
    random.shuffle(files)
    #create training set randomly
    for i in range(int(len(files)*0.8)):
        dest = shutil.copy(cl+'/'+files[i], dest_train+'/'+folder+'/'+files[i])    # copy an image in the training set
        dataset_split["training"][folder].append(files[i])           # insert name of file used in training set
    #create validation set randomly
    for j in range(i+1, len(files)):
        dest = shutil.copy(cl+'/'+files[j], dest_valid+'/'+folder+'/'+files[j])    # copy an image in the validation set
        dataset_split["validation"][folder].append(files[j])           # insert name of file used in validation set


# create the json file using the dictionary dataset_split
with open('dataset_split.json', 'w') as fp:
    json.dump(dataset_split, fp)

In [ ]:
# Create generators to read images from dataset directory
# -------------------------------------------------------
from collections import Counter

dataset_dir = '/kaggle/working/tmp'

# Batch size
bs = 8

# img shape
img_h = 256
img_w = 256

num_classes = 20

classes = ['owl',              # 0
           'galaxy',           # 1
           'lightning',        # 2
           'wine-bottle',      # 3
           't-shirt',          # 4
           'waterfall',        # 5
           'sword',            # 6
           'school-bus',       # 7
           'calculator',       # 8
           'sheet-music',      # 9
           'airplanes',        # 10
           'lightbulb',        # 11
           'skyscraper',       # 12
           'mountain-bike',    # 13
           'fireworks',        # 14
           'computer-monitor', # 15
           'bear',             # 16
           'grand-piano',      # 17
           'kangaroo',         # 18
           'laptop']           # 19

# Training
training_dir = os.path.join(dataset_dir, 'training')
train_gen = train_data_gen.flow_from_directory(training_dir,
                                               batch_size=bs,
                                               target_size=(img_h, img_w),
                                               classes=classes,
                                               class_mode='categorical',
                                               shuffle=True,
                                               seed=SEED)  # targets are directly converted into one-hot vectors

# compute the class weights in order to balance loss during training
counter = Counter(train_gen.classes)                          
max_val = float(max(counter.values()))       
class_weights = {class_id : max_val/num_images for class_id, num_images in counter.items()}   
#print(class_weights)


# Validation
validation_dir = os.path.join(dataset_dir, 'validation')
valid_gen = valid_data_gen.flow_from_directory(validation_dir,
                                               batch_size=bs, 
                                               target_size=(img_h, img_w),
                                               classes=classes,
                                               class_mode='categorical',
                                               shuffle=False,
                                               seed=SEED)

In [ ]:
# Create Dataset objects
# ----------------------

# Training
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

train_dataset = train_dataset.repeat()

# Validation
# ----------
valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

# Repeat
valid_dataset = valid_dataset.repeat()

# TRANSFER LEARNING

In [ ]:
# Load Model
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input

#resnet_weights_path = '../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

# VGG16
#vgg = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))
#vgg = MobileNet(weights='imagenet',include_top=False)

# InceptionV3
inception = tf.keras.applications.InceptionV3(weights='imagenet', 
                                include_top=False, 
                                input_shape=(img_h, img_w,3))

In [ ]:
#vgg.summary()
#vgg.layers

In [ ]:
'''
# freeze layers in VGG16
finetuning = True
if finetuning:
    freeze_until = 15
    
    for layer in vgg.layers[:freeze_until]:
        layer.trainable = False
else:
    vgg.trainable = False
'''

# freeze layers in Inception
count = 1
for layer in inception.layers:
    if count < 260:
        layer.trainable = False
    else:
        layer.trainable = True
    count = count + 1

In [ ]:
'''
# model for Transfer Learning done with VGG16
model = tf.keras.Sequential()
model.add(vgg)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(512, activation='relu', input_dim=(img_h, img_w, 3)))
model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax')) 
'''

# model for Transfer Learning done with Inception
model = tf.keras.models.Sequential()
model.add(inception)
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

# Visualize created model as a table
model.summary()

# Visualize initialized weights
#model.weights

In [ ]:
train_gen.class_indices

## Prepare the model for training

In [ ]:
# Optimization params
# -------------------

# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 2e-5
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Validation metrics
# ------------------

metrics = ['accuracy']
# ------------------

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
import os
from datetime import datetime

# from tensorflow.compat.v1 import ConfigProto
# from tensorflow.compat.v1 import InteractiveSession

# config = ConfigProto()
# config.gpu_options.allow_growth = True
# session = InteractiveSession(config=config)

# Get current working directory
cwd = os.getcwd()

exps_dir = os.path.join(cwd, 'classification_experiments')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = 'CNN'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    

callbacks = []
'''
# Model checkpoint
# ----------------
ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)
    
ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'), 
                                                   save_weights_only=True)  # False to save the model directly
callbacks.append(ckpt_callback)

# Visualize Learning on Tensorboard
# ---------------------------------
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)

# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=1)  # if 1 shows weights histograms
callbacks.append(tb_callback)

# Early Stopping
# --------------
early_stop = False
if early_stop:
    es_callback = tf.keras.callback.EarlyStopping(monitor='val_loss', patience=10)
    callbacks.append(es_callback)
'''

model.fit(x=train_dataset,
          epochs=30,  #### set repeat in training dataset
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen),
          class_weight=class_weights)

In [ ]:
#------ TEST --------

from PIL import Image
from matplotlib.pyplot import imshow

# path of images to test
test_dir = '/kaggle/input/ann-and-dl-image-classification/Classification_Dataset/test'

results = {}

for img_name in os.listdir(test_dir):
    img_path = os.path.join(test_dir, img_name)

    img = tf.io.read_file(img_path)
    #image = Image.open(str(img_path))

    img = tf.image.decode_jpeg(img, channels=3)
    img.set_shape([None, None, 3])
    img = tf.image.resize(img, (img_h, img_w))
    img /= 255
    #imshow(np.asarray(img))
    img = np.expand_dims(img, 0) # make 'batch' of 1
    
    # compute the prediction for each class
    pred = model.predict(img)
    
    # take the predicted class
    y = pred.argmax(axis=-1)[0]
    
    # upload on a dictionary the image with its predicted label
    results[img_name] = y


#print(results)

In [ ]:
# save the predictions in a .csv file

def create_csv(results, results_dir='./'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')

create_csv(results)
            